# Trabajo Práctico - Procesamiento del Lenguaje Natural con Transformers

**Alumno:** Carmen Rodríguez

**Materia:** Técnicas del Procesamiento del Procesamiento del Habla  

**Fecha:** Octubre 2025


# Aplicaciones Prácticas de NLP con Transformers

## Guía de Ejercicios Progresivos

---

## Introducción

En este cuaderno vas a trabajar con aplicaciones reales de Procesamiento de Lenguaje Natural (NLP) utilizando modelos Transformer preentrenados. Cada ejercicio representa un caso de uso concreto que se implementa en empresas y organizaciones de Argentina y el mundo.

Los ejercicios están organizados de manera progresiva:

1. **Nivel Básico:** Moderación automática de comentarios en redes sociales
2. **Nivel Intermedio:** Extracción de información de currículums (RRHH)
3. **Nivel Avanzado:** Chatbot de soporte técnico con respuestas automáticas
4. **Desafío Autónomo:** Análisis integral de reseñas de restaurantes

### ¿Qué vas a aprender?

- A usar modelos preentrenados de Hugging Face para tareas reales
- A comprender cómo funcionan los pipelines de NLP en la práctica
- A modificar y experimentar con código para resolver problemas concretos
- A evaluar la calidad de las predicciones de los modelos
- A trabajar de manera autónoma en un problema real completo

### Requisitos previos

- Conocimientos básicos de Python
- Familiaridad con conceptos de Machine Learning
- Haber visto los cuadernos introductorios sobre clasificación de sentimientos

---

## Instalación de dependencias

Ejecutá la siguiente celda para instalar las librerías necesarias:

In [1]:
# Instalación de librerías (ejecutar solo una vez)
!pip install transformers torch pandas -q

---

## Ejercicio 1: Moderación de Comentarios en Redes Sociales

### Contexto del problema

Imaginate que trabajás en el equipo de redes sociales de una empresa argentina de delivery de comida. Recibís cientos de comentarios diarios en Instagram y Facebook. Necesitás un sistema que identifique automáticamente los comentarios negativos o críticos para que el equipo de atención al cliente pueda responderlos con prioridad.

### Aplicación real

Este tipo de sistemas se usan en:
- Redes sociales de empresas (PedidosYa, Rappi, Mercado Libre)
- Plataformas de e-commerce
- Servicios de atención al cliente automatizada

### ¿Qué vamos a hacer?

1. Cargar un modelo de análisis de sentimientos en español
2. Analizar comentarios reales de clientes
3. Clasificarlos automáticamente (positivo, negativo, neutral)
4. Identificar cuáles requieren atención urgente

---

### Paso 1: Importar librerías y cargar el modelo

In [2]:
# Importamos el pipeline de Hugging Face permite cargar modelos preentrenados
from transformers import pipeline
# Importamos pandas para organizar los resultados en tablas
import pandas as pd

# Cargamos un modelo de análisis de sentimientos en español
# Este modelo fue entrenado por "Finite Automata" sobre datos de redes sociales en español
# Devuelve una etiqueta (POS, NEG o NEU) y un nivel de confianza
clasificador = pipeline(
    "text-classification",
    model="finiteautomata/beto-sentiment-analysis"
)

print("Modelo cargado correctamente.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


Modelo cargado correctamente.


🧪 ¿Qué hace este modelo?

Está basado en BETO (el BERT entrenado en español).

Fue ajustado ("fine-tuned") para analizar comentarios reales de redes sociales.

Clasifica textos en tres clases:

POS: Positivo

NEG: Negativo

NEU: Neutral

### Paso 2: Definir comentarios de ejemplo

Acá tenemos comentarios típicos que podría recibir una empresa de delivery:

Incluimos ejemplos positivos, negativos y neutrales para probar el modelo

In [4]:
comentarios = [
    "La comida llegó rapidísimo y estaba caliente. El repartidor re amable, excelente servicio!",
    "Tardaron DOS HORAS para traer una pizza fría. Pésimo, no pido más acá.",
    "Todo bien, llegó en tiempo y forma. Nada para destacar.",
    "Me encanta este servicio, siempre cumplen. Los re banco!",
    "Se equivocaron con mi pedido OTRA VEZ. Ya es la tercera vez que pasa, un desastre total.",

    # Positivos (explícitos)
    "Excelente atención y llegó todo calentito 😋",
    "¡Sos lo más PedidosYa! Siempre me salvan ❤️",
    "De diez, el chico súper respetuoso y llegó rapidísimo.",

    # Negativos (explícitos)
    "Una vergüenza, pedí a las 8 y llegó a las 10:30. Nunca más.",
    "Pedí empanadas y me trajeron sushi, no entiendo nada.",
    "¡Un desastre! Encima no atienden el teléfono.",

    # Ambiguos o sarcásticos
    "Genial, como siempre me traen lo que NO pedí 😒",
    "No tardaron tanto esta vez, solo 1 hora y media 👏",
    "Al menos llegó... algo es algo.",

    # Neutrales o difíciles de clasificar
    "Llegó dentro del horario estimado.",
    "Comida normal. Nada wow, nada malo.",
    "No sé si fue suerte o mejoraron el servicio.",

    # Jerga / lenguaje coloquial argentino
    "Zafó, pero podrían meterle más onda al packaging.",
    "Está joya, me re cebó la atención del chabón.",
    "Pintó pedir y la clavaron justo. Bien ahí.",

    # Emojis / escritura informal
    "😡😡😡 tardaron muchísimo otra vez",
    "😍 la comida, 10/10 como siempre!!",
    "no sé... 🤷🏻‍♀️ vino bien, pero no me mató tampoco"
]

### Paso 3: Analizar los comentarios

In [6]:
# Procesamos todos los comentarios
resultados = clasificador(comentarios)

# Creamos un DataFrame para visualizar mejor los resultados
df_resultados = pd.DataFrame({
    'Comentario': comentarios,
    'Sentimiento': [r['label'] for r in resultados],
    'Confianza': [round(r['score'], 3) for r in resultados]
})
df_resultados




,Comentario,Sentimiento,Confianza
0,La comida llegó rapidísimo y estaba caliente. ...,POS,0.998
1,Tardaron DOS HORAS para traer una pizza fría. ...,NEG,0.999
2,"Todo bien, llegó en tiempo y forma. Nada para ...",POS,0.997
3,"Me encanta este servicio, siempre cumplen. Los...",POS,0.999
4,Se equivocaron con mi pedido OTRA VEZ. Ya es l...,NEG,0.999
5,Excelente atención y llegó todo calentito 😋,NEU,0.539
6,¡Sos lo más PedidosYa! Siempre me salvan ❤️,POS,0.999
7,"De diez, el chico súper respetuoso y llegó rap...",POS,0.984
8,"Una vergüenza, pedí a las 8 y llegó a las 10:3...",NEG,0.999
9,"Pedí empanadas y me trajeron sushi, no entiend...",NEG,0.930


### Paso 4: Identificar comentarios que requieren atención urgente

In [8]:

# Mostramos los resultados ordenados por tipo de sentimiento
df_resultados.sort_values(by='Sentimiento', ascending=False).reset_index(drop=True)


# Filtramos solo los comentarios negativos
comentarios_urgentes = df_resultados[df_resultados['Sentimiento'] == 'NEG']

print("COMENTARIOS QUE REQUIEREN ATENCIÓN URGENTE:")
print("="*60)
for idx, fila in comentarios_urgentes.iterrows():
    print(f"\n[{idx+1}] {fila['Comentario']}")
    print(f"    Confianza: {fila['Confianza']*100:.1f}%")

COMENTARIOS QUE REQUIEREN ATENCIÓN URGENTE:

[2] Tardaron DOS HORAS para traer una pizza fría. Pésimo, no pido más acá.
    Confianza: 99.9%

[5] Se equivocaron con mi pedido OTRA VEZ. Ya es la tercera vez que pasa, un desastre total.
    Confianza: 99.9%

[9] Una vergüenza, pedí a las 8 y llegó a las 10:30. Nunca más.
    Confianza: 99.9%

[10] Pedí empanadas y me trajeron sushi, no entiendo nada.
    Confianza: 93.0%

[11] ¡Un desastre! Encima no atienden el teléfono.
    Confianza: 99.9%

[19] Está joya, me re cebó la atención del chabón.
    Confianza: 93.0%

[21] 😡😡😡 tardaron muchísimo otra vez
    Confianza: 99.9%


In [12]:
# Agregamos una columna que indique si el comentario necesita atención
# Si el sentimiento es NEGATIVO => requiere atención prioritaria
df_resultados['Atención urgente'] = df_resultados['Sentimiento'].apply(lambda s: '✅' if s == 'NEG' else '')

#df_resultados

df_resultados.sort_values(by='Sentimiento', ascending=True).reset_index(drop=True)

,Comentario,Sentimiento,Confianza,Atención urgente
0,Tardaron DOS HORAS para traer una pizza fría. ...,NEG,0.999,✅
1,😡😡😡 tardaron muchísimo otra vez,NEG,0.999,✅
2,Se equivocaron con mi pedido OTRA VEZ. Ya es l...,NEG,0.999,✅
3,"Está joya, me re cebó la atención del chabón.",NEG,0.930,✅
4,"Una vergüenza, pedí a las 8 y llegó a las 10:3...",NEG,0.999,✅
5,"Pedí empanadas y me trajeron sushi, no entiend...",NEG,0.930,✅
6,¡Un desastre! Encima no atienden el teléfono.,NEG,0.999,✅
7,"no sé... 🤷🏻‍♀️ vino bien, pero no me mató tampoco",NEU,0.997,
8,Pintó pedir y la clavaron justo. Bien ahí.,NEU,0.994,
9,Excelente atención y llegó todo calentito 😋,NEU,0.539,


### Actividad práctica

Ahora es tu turno. Realizá las siguientes tareas:

1. **Agregá 3 comentarios nuevos** a la lista (pueden ser inventados o reales)
2. **Ejecutá nuevamente** las celdas de análisis
3. **Respondé:**
   - ¿El modelo clasificó correctamente tus comentarios?
   - ¿Hubo algún comentario que esperabas que clasifique diferente?
   - ¿Qué pasa si escribís un comentario con ironía? (probá con algo como "Genial, 3 horas esperando, lo mejor!")

Escribí tus comentarios nuevos en la siguiente celda:

In [14]:
# TUS COMENTARIOS ACÁ
mis_comentarios = [
    "¡Increíble atención! Llegó justo a tiempo y estaba todo perfecto. ✨",     # POSITIVO CLARO
    "Genial, 3 horas esperando, lo mejor que me pasó hoy 🙃",                  # SARCASMO / NEGATIVO
    "No fue tan malo como la otra vez, pero tampoco excelente.",              # NEUTRO / AMBIGUO
]



# Analizamos
mis_resultados = clasificador(mis_comentarios)

# Mostramos
df_mis_resultados = pd.DataFrame({
    'Comentario': mis_comentarios,
    'Sentimiento': [r['label'] for r in mis_resultados],
    'Confianza': [round(r['score'], 3) for r in mis_resultados]
})

df_mis_resultados


### Reflexión sobre el análisis

#Probé con un comentario sarcástico ("Genial, 3 horas esperando, lo mejor que me pasó hoy 🙃") y el modelo lo clasificó como POSITIVO con 99.6% de confianza.
#Esto indica que el modelo no detecta sarcasmo, lo cual es esperable, ya que los modelos preentrenados generalmente necesitan más ejemplos explícitos de ironía para poder aprender este tipo de lenguaje.

#El resto de los comentarios fueron correctamente clasificados en función de sus palabras clave.
#Podría mejorarse el desempeño con:
#- Más datos de entrenamiento con sarcasmo
#- Análisis de emojis o puntuación
#- Modelos con mayor capacidad de comprensión contextual


,Comentario,Sentimiento,Confianza
0,¡Increíble atención! Llegó justo a tiempo y es...,POS,0.998
1,"Genial, 3 horas esperando, lo mejor que me pas...",POS,0.996
2,"No fue tan malo como la otra vez, pero tampoco...",POS,0.958


---

## Ejercicio 2: Extracción Automática de Información de CVs

### Contexto del problema

Trabajás en el área de Recursos Humanos de una consultora argentina. Recibís decenas de CVs por día y necesitás extraer rápidamente información clave: nombres de candidatos, empresas donde trabajaron, universidades, ciudades, y tecnologías que manejan.

### Aplicación real

Este tipo de sistemas se usan en:
- Plataformas de empleo (LinkedIn, Bumeran, Computrabajo)
- Sistemas de ATS (Applicant Tracking Systems)
- Automatización de procesos de RRHH

### ¿Qué vamos a hacer?

1. Cargar un modelo de Reconocimiento de Entidades Nombradas (NER)
2. Procesar un fragmento de CV
3. Extraer automáticamente: personas, organizaciones, ubicaciones
4. Organizar la información de manera estructurada

---

### Paso 1: Cargar el modelo de NER

In [15]:
# Importamos el pipeline de Hugging Face, en este caso no es necesaria porque lo hicimos con anterioridad
#from transformers import pipeline


# Cargamos un modelo de NER (Reconocimiento de Entidades Nombradas) específico para español
# Este modelo fue entrenado para reconocer personas, organizaciones y lugares (fine-tuneado para tareas en español como CVs, noticias, etc.)

extractor_ner = pipeline(
    "ner",
    model="mrm8488/bert-spanish-cased-finetuned-ner",
    aggregation_strategy="simple"  # Agrupa tokens de la misma entidad
)

print("Modelo NER cargado correctamente.")

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


Modelo NER cargado correctamente.


¿Qué es NER (Named Entity Recognition)?

Es una técnica de NLP que permite identificar y clasificar automáticamente entidades nombradas en texto, como:

PER: Personas

ORG: Organizaciones

LOC: Localizaciones (ciudades, países)

MISC: Otros (tecnologías, fechas, etc.)

### Paso 2: Definir un fragmento de CV

Este es un ejemplo típico de cómo un candidato describe su experiencia:

In [16]:
cv_texto = """María Fernández es una desarrolladora full-stack con 5 años de experiencia.
Estudió Ingeniería en Sistemas en la Universidad de Buenos Aires (UBA) y se graduó en 2018.
Trabajó como desarrolladora backend en Mercado Libre durante 3 años, donde lideró la migración
de sistemas legacy a microservicios. Posteriormente se unió a Globant como tech lead,
coordinando equipos distribuidos entre Buenos Aires y Córdoba. Tiene experiencia en Python,
Java, Docker y Kubernetes. Actualmente reside en Palermo, Ciudad de Buenos Aires."""

print("CV a procesar:")
print(cv_texto)

CV a procesar:
María Fernández es una desarrolladora full-stack con 5 años de experiencia.
Estudió Ingeniería en Sistemas en la Universidad de Buenos Aires (UBA) y se graduó en 2018.
Trabajó como desarrolladora backend en Mercado Libre durante 3 años, donde lideró la migración
de sistemas legacy a microservicios. Posteriormente se unió a Globant como tech lead,
coordinando equipos distribuidos entre Buenos Aires y Córdoba. Tiene experiencia en Python,
Java, Docker y Kubernetes. Actualmente reside en Palermo, Ciudad de Buenos Aires.


### Paso 3: Extraer entidades del CV

In [17]:
# Procesamos el texto del CV
entidades = extractor_ner(cv_texto)

# Creamos un DataFrame para visualizar mejor
df_entidades = pd.DataFrame(entidades)

# Mostramos solo las columnas relevantes
df_entidades[['entity_group', 'word', 'score']].round(3)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,entity_group,word,score
0,PER,María Fernández,1.000
1,MISC,Ingeniería en Sistemas,0.998
2,ORG,Universidad de Buenos Aires,0.998
3,ORG,UBA,0.983
4,MISC,Mercado Libre,0.789
5,ORG,Glo,1.000
6,ORG,##bant,0.821
7,LOC,Buenos Aires,1.000
8,LOC,Córdoba,1.000
9,MISC,Py,0.999


### Paso 4: Organizar la información por categoría

Ahora vamos a separar las entidades por tipo para que sea más fácil revisarlas:

In [21]:
# Diccionario para organizar por tipo de entidad
info_extraida = {
    'PER': [],  # Personas
    'ORG': [],  # Organizaciones
    'LOC': []   # Ubicaciones
}

# Clasificamos cada entidad
for entidad in entidades:
    tipo = entidad['entity_group']
    palabra = entidad['word']
    if tipo in info_extraida:
        info_extraida[tipo].append(palabra)


# Mostramos la información de forma estructurada
print("📄 INFORMACIÓN EXTRAÍDA DEL CV")
print("="*60)
print(f"\n👤 CANDIDATO/A: {', '.join(info_extraida['PER'])}")
print(f"\n🏢 EMPRESAS/INSTITUCIONES: {', '.join(info_extraida['ORG'])}")
print(f"\n📍 UBICACIONES: {', '.join(info_extraida['LOC'])}")
#me hace una división rara en Globant

📄 INFORMACIÓN EXTRAÍDA DEL CV

👤 CANDIDATO/A: María Fernández

🏢 EMPRESAS/INSTITUCIONES: Universidad de Buenos Aires, UBA, Glo, ##bant

📍 UBICACIONES: Buenos Aires, Córdoba, Palermo, Ciudad de Buenos Aires


### Actividad práctica

Ahora es tu turno. Realizá las siguientes tareas:

1. **Escribí un fragmento de CV ficticio** (o usá el tuyo si querés) con:
   - Nombre de la persona
   - Al menos 2 empresas o instituciones educativas
   - Al menos 2 ubicaciones (ciudades, barrios, países)

2. **Ejecutá el análisis** y verificá qué entidades detectó el modelo

3. **Respondé:**
   - ¿El modelo identificó correctamente todas las entidades?
   - ¿Hubo alguna entidad que no detectó? ¿Por qué creés que pasó?
   - ¿Detectó alguna entidad incorrectamente?

Escribí tu CV ficticio en la siguiente celda:

In [25]:
# TU CV FICTICIO ACÁ
mi_cv = """QA Engineer Carmen Rodríguez con experiencia en desarrollo de casos de uso, ejecución de casos de prueba y monitoreo de resultados. Habilidades en pruebas de frontend, base de datos, aplicaciones móviles, servicios web y API en entornos ágiles.
Experiencia en planificación de proyectos de diversas empresas como PDVSA, Accenture y NGL Nitú en países como Venezuela, Argentina y 100$ remoto, coordinación de tareas entre equipos multidisciplinarios y facilitación de ceremonias Scrum. Destrezas en análisis, resolución de problemas, comunicación y trabajo en equipo.
Experto en creación y mantenimiento de documentos de prueba para garantizar la calidad del software. Mis puntos fuertes CliftonStrengths 34 son aprendiz, individualizado, realizado, positivo y sociable."""

# Procesamos
mis_entidades = extractor_ner(mi_cv)

# Mostramos
df_mis_entidades = pd.DataFrame(mis_entidades)
print("Entidades detectadas:")
print(df_mis_entidades[['entity_group', 'word', 'score']].round(3))

# Organizamos
mi_info = {'PER': [], 'ORG': [], 'LOC': []}
for e in mis_entidades:
    if e['entity_group'] in mi_info:
        mi_info[e['entity_group']].append(e['word'])

print("\n" + "="*60)
print(f"CANDIDATO/A: {', '.join(mi_info['PER'])}")
print(f"EMPRESAS/INSTITUCIONES: {', '.join(mi_info['ORG'])}")
print(f"UBICACIONES: {', '.join(mi_info['LOC'])}")

Entidades detectadas:
   entity_group                 word  score
0          MISC                   QA  0.863
1           ORG               Engine  0.549
2          MISC                 ##er  0.746
3           PER     Carmen Rodríguez  0.947
4          MISC                   AP  0.998
5          MISC                  ##I  0.754
6           ORG                   PD  1.000
7           ORG                ##VSA  0.968
8           ORG                   Ac  1.000
9           ORG            ##centure  0.744
10          ORG                    N  1.000
11          ORG            ##GL Nitú  0.995
12          LOC            Venezuela  1.000
13          LOC            Argentina  1.000
14         MISC                Scrum  0.928
15         MISC  CliftonStrengths 34  0.928

CANDIDATO/A: Carmen Rodríguez
EMPRESAS/INSTITUCIONES: Engine, PD, ##VSA, Ac, ##centure, N, ##GL Nitú
UBICACIONES: Venezuela, Argentina


---

## Ejercicio 3: Chatbot de Soporte Técnico Automático

### Contexto del problema

Trabajás en una empresa que vende electrodomésticos online. Los clientes suelen hacer preguntas frecuentes sobre garantías, envíos y devoluciones. Querés automatizar las respuestas a estas consultas usando un sistema de Question Answering (QA) que pueda responder basándose en la información de tus políticas.

### Aplicación real

Este tipo de sistemas se usan en:
- Chatbots de atención al cliente (WhatsApp, web)
- Sistemas de FAQ automáticas
- Asistentes virtuales corporativos

### ¿Qué vamos a hacer?

1. Cargar un modelo de Question Answering en español
2. Definir un contexto (políticas de la empresa)
3. Hacer preguntas sobre ese contexto
4. Generar respuestas automáticas

---

### Paso 1: Cargar los modelos necesarios

In [29]:
from transformers import pipeline

# Modelo multilingüe para QA (funciona en español)
qa_modelo = pipeline(
    "question-answering",
    model="deepset/xlm-roberta-base-squad2"
)

print("✅ Modelo de QA cargado correctamente.")


config.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cpu


✅ Modelo de QA cargado correctamente.


### Paso 2: Definir el contexto (políticas de la empresa)

Este es el documento que el chatbot va a usar para responder preguntas:

In [30]:
politicas_empresa = """POLÍTICAS DE GARANTÍA Y DEVOLUCIONES - ELECTROHOGAR S.A.

GARANTÍA: Todos nuestros productos tienen garantía oficial de 12 meses contra defectos
de fábrica. La garantía comienza a contar desde la fecha de recepción del producto.
Para hacer válida la garantía, el cliente debe presentar la factura de compra original
y el producto no debe tener daños físicos causados por mal uso.

ENVÍOS: Realizamos envíos a todo el país. En Capital Federal y Gran Buenos Aires,
el envío demora entre 24 y 48 horas hábiles. Para el interior del país, el tiempo
de entrega es de 3 a 7 días hábiles. El envío es gratuito para compras superiores
a $50.000. Para montos menores, se cobra un adicional de $2.500.

DEVOLUCIONES: El cliente tiene 10 días corridos desde la recepción del producto para
solicitar una devolución si el artículo no cumple con sus expectativas. El producto
debe estar sin usar, en su empaque original y con todos sus accesorios. El cliente
debe hacerse cargo del costo de envío de devolución. Una vez recibido y verificado
el producto, reintegramos el 100% del dinero en un plazo de 15 días hábiles.
"""

print("Contexto cargado:")
print(politicas_empresa)

Contexto cargado:
POLÍTICAS DE GARANTÍA Y DEVOLUCIONES - ELECTROHOGAR S.A.

GARANTÍA: Todos nuestros productos tienen garantía oficial de 12 meses contra defectos
de fábrica. La garantía comienza a contar desde la fecha de recepción del producto.
Para hacer válida la garantía, el cliente debe presentar la factura de compra original
y el producto no debe tener daños físicos causados por mal uso.

ENVÍOS: Realizamos envíos a todo el país. En Capital Federal y Gran Buenos Aires,
el envío demora entre 24 y 48 horas hábiles. Para el interior del país, el tiempo
de entrega es de 3 a 7 días hábiles. El envío es gratuito para compras superiores
a $50.000. Para montos menores, se cobra un adicional de $2.500.

DEVOLUCIONES: El cliente tiene 10 días corridos desde la recepción del producto para
solicitar una devolución si el artículo no cumple con sus expectativas. El producto
debe estar sin usar, en su empaque original y con todos sus accesorios. El cliente
debe hacerse cargo del costo de envío

### Paso 3: Hacer preguntas al sistema

In [47]:
# Definimos preguntas típicas de clientes
preguntas = [
    "¿Cuánto dura la garantía?",
    "¿Cuánto tarda el envío a Capital Federal?",
    "¿Cuántos días tengo para devolver un producto?",
    "¿Qué necesito para hacer válida la garantía?",
    "¿El envío es gratis?",
    "¿Cuál es el plazo para hacer una devolución?"

]

# Procesamos cada pregunta
print("RESPUESTAS AUTOMÁTICAS DEL CHATBOT")
print("="*60)

for pregunta in preguntas:
    respuesta = qa_modelo(question=pregunta, context=politicas_empresa)
    print(f"\nPREGUNTA: {pregunta}")
    print(f"RESPUESTA: {respuesta['answer']}")
    print(f"Confianza: {respuesta['score']*100:.1f}%")
    print("-"*60)

RESPUESTAS AUTOMÁTICAS DEL CHATBOT

PREGUNTA: ¿Cuánto dura la garantía?
RESPUESTA:  12 meses
Confianza: 42.0%
------------------------------------------------------------

PREGUNTA: ¿Cuánto tarda el envío a Capital Federal?
RESPUESTA:  24 y 48 horas hábiles.
Confianza: 48.3%
------------------------------------------------------------

PREGUNTA: ¿Cuántos días tengo para devolver un producto?
RESPUESTA:  10 días
Confianza: 7.2%
------------------------------------------------------------

PREGUNTA: ¿Qué necesito para hacer válida la garantía?
RESPUESTA:  presentar la factura de compra original
Confianza: 19.4%
------------------------------------------------------------

PREGUNTA: ¿El envío es gratis?
RESPUESTA:  para compras superiores
a $50.000.
Confianza: 21.4%
------------------------------------------------------------

PREGUNTA: ¿Cuál es el plazo para hacer una devolución?
RESPUESTA:  10 días corridos
Confianza: 19.8%
------------------------------------------------------------


### Paso 4: Crear una función interactiva de chatbot

Vamos a crear una función que simule un chatbot completo:

In [59]:
def chatbot_soporte(pregunta, contexto=politicas_empresa):
    """
    Función extendida con depuración: muestra respuesta y confianza.
    """
    resultado = qa_modelo(question=pregunta, context=contexto)

    print("📍 Pregunta:", pregunta)
    print("📌 Respuesta:", resultado['answer'])
    print("🤖 Confianza:", round(resultado['score'], 3))

    # Retorno normal si querés seguir usándola como chatbot
    if resultado['score'] > 0.25:
        return f"💬 {resultado['answer']}"
    else:
        return "⚠️ No encontré una respuesta clara."


In [60]:
chatbot_soporte("¿En cuánto tiempo puedo devolver un producto si no me gusta?")


📍 Pregunta: ¿En cuánto tiempo puedo devolver un producto si no me gusta?
📌 Respuesta:  10 días corridos
🤖 Confianza: 0.283


'💬  10 días corridos'

### Actividad práctica

Ahora es tu turno. Realizá las siguientes tareas:

1. **Escribí 3 preguntas nuevas** que un cliente podría hacer sobre las políticas
2. **Probá el chatbot** con esas preguntas
3. **Modificá el contexto** (politicas_empresa) agregando información nueva, por ejemplo:
   - Formas de pago aceptadas
   - Horarios de atención
   - Información sobre instalación de productos
4. **Respondé:**
   - ¿Las respuestas fueron precisas?
   - ¿Hubo alguna pregunta que el modelo no pudo responder bien?
   - ¿Qué pasa si hacés una pregunta sobre algo que NO está en el contexto?

Usá las siguientes celdas para experimentar:

In [66]:
# MODIFICÁ EL CONTEXTO ACÁ (agregá información nueva)
mi_contexto_ampliado = """
Todos nuestros productos tienen una garantía de 12 meses desde la fecha de compra.
Si el producto presenta fallas técnicas dentro de ese período, el cliente puede solicitar la reparación sin costo adicional.

Los envíos se realizan a todo el país en un plazo de 3 a 7 días hábiles.
En caso de demoras, el cliente puede consultar el estado del envío en nuestro sitio web usando su número de pedido.

Para devolver un producto, el cliente tiene hasta 10 días corridos desde la recepción del pedido.
El producto debe estar en su empaque original y sin signos de uso.
La devolución se realiza sin cargo si el producto llegó dañado o incorrecto.

Aceptamos pagos con tarjeta de crédito, débito, transferencia bancaria y billeteras virtuales como Mercado Pago.

Nuestro horario de atención al cliente es de lunes a viernes, de 9 a 18 hs.

La instalación de productos como aires acondicionados o cocinas está incluida en el precio si el cliente reside en Ciudad de Buenos Aires o Gran Buenos Aires.
"""

# Probá con una pregunta sobre la información nueva que agregaste
print(chatbot_soporte("¿cuaél es el horario de atención?", contexto=mi_contexto_ampliado))
print(chatbot_soporte("¿cuaél es el plazo de envíos?", contexto=mi_contexto_ampliado))
print(chatbot_soporte("¿Aceptan tarjeta?", contexto=mi_contexto_ampliado))

📍 Pregunta: ¿cuaél es el horario de atención?
📌 Respuesta:  lunes a viernes,
🤖 Confianza: 0.421
💬  lunes a viernes,
📍 Pregunta: ¿cuaél es el plazo de envíos?
📌 Respuesta:  3 a 7 días hábiles.
🤖 Confianza: 0.595
💬  3 a 7 días hábiles.
📍 Pregunta: ¿Aceptan tarjeta?
📌 Respuesta:  pagos con tarjeta de crédito,
🤖 Confianza: 0.116
⚠️ No encontré una respuesta clara.


In [74]:
# TUS PREGUNTAS ACÁ
mis_preguntas = [
"¿Cuál es el tiempo de entrega?",
"¿Hacen instalaciones en Córdoba?"
]

# Probamos el chatbot con tus preguntas
for pregunta in mis_preguntas:
    print("\n" + "="*60)
    print(chatbot_soporte(pregunta))


📍 Pregunta: ¿Cuál es el tiempo de entrega?
📌 Respuesta:  3 a 7 días hábiles.
🤖 Confianza: 0.507
💬  3 a 7 días hábiles.

📍 Pregunta: ¿Hacen instalaciones en Córdoba?
📌 Respuesta:  Realizamos envíos a todo el país.
🤖 Confianza: 0.0
⚠️ No encontré una respuesta clara.


In [75]:
# Probamos el chatbot con tus preguntas
for pregunta in mis_preguntas:
    print("\n" + "="*60)
    print(f"💬 Cliente pregunta: {pregunta}")
    print(chatbot_soporte(pregunta, contexto=politicas_empresa))



💬 Cliente pregunta: ¿Cuál es el tiempo de entrega?
📍 Pregunta: ¿Cuál es el tiempo de entrega?
📌 Respuesta:  3 a 7 días hábiles.
🤖 Confianza: 0.507
💬  3 a 7 días hábiles.

💬 Cliente pregunta: ¿Hacen instalaciones en Córdoba?
📍 Pregunta: ¿Hacen instalaciones en Córdoba?
📌 Respuesta:  Realizamos envíos a todo el país.
🤖 Confianza: 0.0
⚠️ No encontré una respuesta clara.


---

## Ejercicio 4: Desafío Autónomo - Análisis de Reseñas de Restaurantes

### Contexto del problema

Sos el encargado de marketing digital de una cadena de restaurantes porteña. Querés implementar un sistema inteligente que procese automáticamente las reseñas que los clientes dejan en Google Maps y redes sociales para:

1. Identificar si la reseña es positiva, negativa o neutral
2. Extraer información clave: nombres de platos mencionados, ubicaciones de las sucursales, nombres de empleados destacados
3. Responder automáticamente a preguntas frecuentes basándose en el menú y políticas del restaurante

### Aplicación real

Este tipo de sistemas combinados se usan en:
- Gestión de reputación online para cadenas de restaurantes
- Análisis de feedback de clientes en hotelería y turismo
- Sistemas de CRM (Customer Relationship Management) inteligentes
- Plataformas de delivery con análisis de satisfacción del cliente

---

### Tu tarea

**Este ejercicio lo tenés que resolver completamente solo**, aplicando todo lo que aprendiste en los ejercicios anteriores. No hay código de ejemplo, solo las instrucciones.

### Parte 1: Análisis de sentimiento de reseñas (30%)

1. Creá una lista con al menos 5 reseñas ficticias de clientes sobre un restaurante argentino (podés inventarlas o buscar reales)
2. Cargá un modelo de análisis de sentimientos en español (buscá en Hugging Face)
3. Clasificá cada reseña y mostrá los resultados en un DataFrame
4. Identificá cuántas reseñas son positivas, negativas y neutrales (si el modelo lo soporta)

**Pistas:**
- Usá `pipeline("text-classification", model=...)`
- Recordá importar `pandas` para crear el DataFrame
- Modelos sugeridos: `finiteautomata/beto-sentiment-analysis` o `pysentimiento/robertuito-sentiment-analysis`

---

### Parte 2: Extracción de información (40%)

1. Tomá 2 de las reseñas que creaste (las más largas y detalladas)
2. Cargá un modelo de NER en español
3. Extraé todas las entidades nombradas de esas reseñas
4. Organizá la información en categorías (personas, lugares, organizaciones)
5. Bonus: ¿Se mencionan nombres de platos? (Nota: el modelo podría no detectarlos como entidades, reflexioná sobre por qué)

**Pistas:**
- Usá `pipeline("ner", model=..., aggregation_strategy="simple")`
- Modelo sugerido: `mrm8488/bert-spanish-cased-finetuned-ner`
- Recordá iterar sobre los resultados para organizarlos por tipo

---

### Parte 3: Sistema de preguntas y respuestas (30%)

1. Escribí un texto con información del restaurante (menú, horarios, ubicación, políticas de reservas, precios promedio, etc.). Mínimo 4-5 oraciones.
2. Cargá un modelo de Question Answering en español
3. Formulá al menos 4 preguntas que un cliente podría hacer
4. Generá respuestas automáticas usando el modelo
5. Mostrá cada pregunta con su respuesta y el nivel de confianza del modelo

**Pistas:**
- Usá `pipeline("question-answering", model=...)`
- Modelo sugerido: `PlanTL-GOB-ES/roberta-base-bne-sqac`
- La función necesita dos parámetros: `question=` y `context=`

---

### Bonus (opcional): Integración completa

Si terminaste las tres partes, intentá crear una función que:
1. Reciba una reseña de cliente como input
2. Analice el sentimiento
3. Extraiga entidades mencionadas
4. Genere un resumen estructurado

Por ejemplo:
```
RESEÑA: "Fui ayer a la sucursal de Palermo y el mozo Juan me atendió bárbaro..."

ANÁLISIS:
- Sentimiento: POSITIVO (95% confianza)
- Empleado mencionado: Juan
- Sucursal: Palermo
- Recomendación: Enviar agradecimiento personalizado
```

---

### Criterios de evaluación

Evaluá tu propio trabajo considerando:

1. **Funcionalidad (50%):** ¿El código funciona sin errores? ¿Completaste las tres partes?
2. **Calidad de datos (20%):** ¿Las reseñas y preguntas son realistas? ¿El contexto tiene información útil?
3. **Presentación (20%):** ¿Los resultados se muestran de forma clara? ¿Usaste DataFrames o print statements organizados?
4. **Reflexión crítica (10%):** ¿Analizaste la calidad de las predicciones? ¿Identificaste limitaciones?

---

### Espacio para tu solución

Usá las celdas siguientes para resolver el desafío. Podés crear todas las celdas que necesites.


In [76]:
# PARTE 1: ANÁLISIS DE SENTIMIENTO

from transformers import pipeline
import pandas as pd

# Cargamos un modelo preentrenado para análisis de sentimientos en español
clasificador_sentimientos = pipeline(
    "text-classification",
    model="finiteautomata/beto-sentiment-analysis"
)

print("✅ Modelo de análisis de sentimientos cargado correctamente.")




Device set to use cpu


✅ Modelo de análisis de sentimientos cargado correctamente.


In [77]:
reseñas = [
    "La milanesa napolitana estaba espectacular, y la atención de Sofía fue impecable. Volvería mil veces.",
    "Muy caro para la calidad de comida que ofrecen. El bife llegó frío y tardaron 50 minutos.",
    "Normal. Nada sobresaliente. Pedí una empanada y una coca, todo ok pero sin emoción.",
    "El mozo Juan nos hizo pasar una noche increíble. Probamos el ojo de bife y el flan casero: una delicia.",
    "El lugar es lindo, pero el servicio fue lento y nos trajeron mal el pedido. No sé si volvería."
]


In [78]:
# Clasificamos cada reseña
resultados = clasificador_sentimientos(reseñas)

# Convertimos a DataFrame
df_resultados = pd.DataFrame({
    "Reseña": reseñas,
    "Sentimiento": [r["label"] for r in resultados],
    "Confianza": [round(r["score"], 3) for r in resultados]
})

# Mostramos los resultados
print(df_resultados)

# Contamos cuántas hay de cada tipo
conteo = df_resultados["Sentimiento"].value_counts()
print("\nConteo de sentimientos:")
print(conteo)


                                              Reseña Sentimiento  Confianza
0  La milanesa napolitana estaba espectacular, y ...         POS      0.998
1  Muy caro para la calidad de comida que ofrecen...         NEG      0.999
2  Normal. Nada sobresaliente. Pedí una empanada ...         NEG      0.974
3  El mozo Juan nos hizo pasar una noche increíbl...         POS      0.999
4  El lugar es lindo, pero el servicio fue lento ...         NEU      0.894

Conteo de sentimientos:
Sentimiento
POS    2
NEG    2
NEU    1
Name: count, dtype: int64


In [79]:
# PARTE 2: EXTRACCIÓN DE INFORMACIÓN (NER)

from transformers import pipeline
import pandas as pd

# Cargamos el modelo NER en español
extractor_ner = pipeline(
    "ner",
    model="mrm8488/bert-spanish-cased-finetuned-ner",
    aggregation_strategy="simple"
)

print("✅ Modelo NER cargado correctamente.")



Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


✅ Modelo NER cargado correctamente.


In [92]:
reseña_1 = "La milanesa napolitana estaba espectacular, y la atención de Marilyn fue impecable. Volvería mil veces a McDonald's ubicada en Lechería."
reseña_2 = "El mozo Juan nos hizo pasar una noche increíble. Probamos el ojo de bife y el flan casero en la sucursal de la Pnera Rosa en Recoleta."

# Juntamos en una lista
reseñas_largas = [reseña_1, reseña_2]


In [93]:
# Procesamos cada reseña
for idx, texto in enumerate(reseñas_largas, start=1):
    print(f"\n🔍 Procesando Reseña {idx}:")
    entidades = extractor_ner(texto)

    # Organizar por tipo
    info_extraida = {'PER': [], 'ORG': [], 'LOC': []}

    for entidad in entidades:
        tipo = entidad["entity_group"]
        palabra = entidad["word"]
        if tipo in info_extraida:
            info_extraida[tipo].append(palabra)

    # Mostrar resultado
    print("="*50)
    print(f"👤 Personas: {', '.join(info_extraida['PER']) or '—'}")
    print(f"🏢 Organizaciones: {', '.join(info_extraida['ORG']) or '—'}")
    print(f"📍 Ubicaciones: {', '.join(info_extraida['LOC']) or '—'}")
    print("="*50)



🔍 Procesando Reseña 1:
👤 Personas: Marilyn
🏢 Organizaciones: ##onald ' s
📍 Ubicaciones: Lechería

🔍 Procesando Reseña 2:
👤 Personas: Juan
🏢 Organizaciones: Pnera Rosa
📍 Ubicaciones: Recoleta


In [94]:
# PARTE 3: QUESTION ANSWERING
qa_modelo = pipeline(
    "question-answering",
    model="deepset/xlm-roberta-base-squad2"
)

print("✅ Modelo de QA cargado correctamente.")




Device set to use cpu


✅ Modelo de QA cargado correctamente.


In [95]:
contexto_restaurante = """
Nuestro restaurante "La Parrilla Porteña" ofrece una experiencia auténtica de comida argentina.
Estamos ubicados en Palermo, Buenos Aires, y abrimos todos los días de 12:00 a 23:00 hs.
No se requiere reserva previa, pero recomendamos hacerla los fines de semana.
Aceptamos pagos en efectivo, tarjetas de crédito, débito y Mercado Pago.
El menú incluye carnes a la parrilla como bife de chorizo, vacío y entraña,
así como empanadas, provoleta, papas fritas, ensaladas y flan con dulce de leche.
El precio promedio por persona es de $12.000. Ofrecemos menú vegetariano y para celíacos.
"""
print("📝 Contexto cargado.")


📝 Contexto cargado.


In [96]:
preguntas_clientes = [
    "¿Dónde está ubicado el restaurante?",
    "¿Cuál es el horario de atención?",
    "¿Tienen opciones para celíacos?",
    "¿Cuánto cuesta comer por persona?",
    "¿Qué platos tienen en el menú?",
    "¿Puedo pagar con Mercado Pago?"
]


In [97]:
for pregunta in preguntas_clientes:
    resultado = qa_modelo(question=pregunta, context=contexto_restaurante)
    confianza = round(resultado['score'], 3)

    print("\n" + "="*60)
    print(f"📌 Pregunta: {pregunta}")
    print(f"🧠 Respuesta: {resultado['answer']}")
    print(f"✅ Confianza: {confianza}")



📌 Pregunta: ¿Dónde está ubicado el restaurante?
🧠 Respuesta:  Palermo, Buenos Aires,
✅ Confianza: 0.896

📌 Pregunta: ¿Cuál es el horario de atención?
🧠 Respuesta:  12:00 a 23:00 hs.
✅ Confianza: 0.669

📌 Pregunta: ¿Tienen opciones para celíacos?
🧠 Respuesta:  Ofrecemos menú vegetariano
✅ Confianza: 0.254

📌 Pregunta: ¿Cuánto cuesta comer por persona?
🧠 Respuesta:  $12.000.
✅ Confianza: 0.87

📌 Pregunta: ¿Qué platos tienen en el menú?
🧠 Respuesta:  bife de chorizo, vacío y entraña,
✅ Confianza: 0.053

📌 Pregunta: ¿Puedo pagar con Mercado Pago?
🧠 Respuesta: 
Aceptamos pagos en efectivo,
✅ Confianza: 0.217


In [98]:
# BONUS (OPCIONAL): INTEGRACIÓN COMPLETA
from transformers import pipeline

# Análisis de sentimiento
sentiment_model = pipeline(
    "text-classification",
    model="finiteautomata/beto-sentiment-analysis"
)

# Extracción de entidades
ner_model = pipeline(
    "ner",
    model="mrm8488/bert-spanish-cased-finetuned-ner",
    aggregation_strategy="simple"
)




Device set to use cpu
Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [99]:
def analizar_reseña_completa(reseña):
    # 1. Análisis de sentimiento
    resultado_sentimiento = sentiment_model(reseña)[0]
    sentimiento = resultado_sentimiento['label']
    confianza_sent = round(resultado_sentimiento['score'], 3)

    # 2. Extracción de entidades
    entidades = ner_model(reseña)
    entidades_extraidas = {'PER': [], 'LOC': [], 'ORG': []}

    for e in entidades:
        if e['entity_group'] in entidades_extraidas:
            entidades_extraidas[e['entity_group']].append(e['word'])

    # 3. Resumen estructurado
    print("="*60)
    print("🔍 ANÁLISIS DE RESEÑA")
    print(f"📌 Reseña original: {reseña}")
    print(f"🧠 Sentimiento: {sentimiento} ({confianza_sent*100}%)")

    print(f"👤 Empleados mencionados: {', '.join(entidades_extraidas['PER']) or 'Ninguno'}")
    print(f"📍 Sucursal o ubicación: {', '.join(entidades_extraidas['LOC']) or 'No detectada'}")
    print(f"🏢 Organización: {', '.join(entidades_extraidas['ORG']) or 'No detectada'}")

    # 4. Recomendación simple
    if sentimiento == "POS":
        print("✅ Recomendación: Enviar agradecimiento personalizado.")
    elif sentimiento == "NEG":
        print("⚠️ Recomendación: Escalar al equipo de atención al cliente.")
    else:
        print("🟡 Recomendación: Monitorear, pero no requiere acción inmediata.")
    print("="*60)


In [100]:
reseña_test = "Fui a la sucursal de Recoleta y el mozo Pablo fue muy amable. Pedí vacío con papas y un flan que estaba riquísimo."
analizar_reseña_completa(reseña_test)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


🔍 ANÁLISIS DE RESEÑA
📌 Reseña original: Fui a la sucursal de Recoleta y el mozo Pablo fue muy amable. Pedí vacío con papas y un flan que estaba riquísimo.
🧠 Sentimiento: POS (99.8%)
👤 Empleados mencionados: Pablo
📍 Sucursal o ubicación: Recoleta
🏢 Organización: No detectada
✅ Recomendación: Enviar agradecimiento personalizado.


ANÁLISIS DE ERRORES, DIFICULTADES Y AJUSTES HECHOS



Problemas con el Chatbot (QA)

Síntoma: El modelo no encontraba respuestas claras aunque la información estuviera presente.

Motivo probable:

El modelo deepset/xlm-roberta-base-squad2 tiene un umbral por defecto que puede ser demasiado alto para ciertos textos.

También es sensible a cómo está redactada la pregunta y cómo se organiza el contexto.

✔️ Qué hice?:

Bajar el umbral de score de 0.5 a 0.3 o 0.25.

Reformular preguntas para alinearlas mejor al texto.

Ampliar el contexto agregando formas de pago, horarios, etc.

NER con platos o nombres propios específicos

Los modelos preentrenados de NER no están entrenados para detectar:

Platos de comida (milanesa napolitana, flan con dulce de leche)

Nombres inventados o informales (Panera Rosa)

✔️ Qué hice?

Segmentar las entidades por tipo (PER, ORG, LOC), y analicé las limitaciones del modelo.

Sarcasmo o ambigüedad en análisis de sentimiento

El modelo clasificó un mensaje sarcástico como positivo.

Esto es esperado: los modelos no entienden sarcasmo a menos que sean específicamente fine-tuneados para eso.


### Reflexión personal

Después de completar el ejercicio, respondé estas preguntas:

1. **¿Qué fue lo más difícil del ejercicio?**
Lograr que el chatbot comprendiera mis preguntas correctamente. Tuve que ajustar (bajar) el umbral de confianza, reformular preguntas, y modificar el contexto para lograr que el modelo diera respuestas acertadas.

2. **¿Encontraste alguna limitación en los modelos?**
Sí. Algunos ejemplos:
-No detectan sarcasmo en análisis de sentimientos.
-No reconocen nombres de platos o ubicaciones inventadas si no están en sus datos de entrenamiento.
-En QA, si la redacción del contexto no es precisa o está en otro formato, no detectan la respuesta.

3. **¿Cómo podrías mejorar este sistema para un caso real?**
-Entrenando modelos con ejemplos del dominio específico (fine-tuning).
-Agregando una capa de reglas o diccionarios personalizados.
-Usando embeddings y búsqueda semántica (retrieval) antes de aplicar QA.
-Incorporando modelos que manejen ironía o lenguaje coloquial argentino.

4. **¿Qué otras aplicaciones se te ocurren para estas técnicas en el contexto argentino?**
-Chatbots para atención ciudadana (AFIP, ANSES, PAMI).
-Chatbots educativos para adolescentes, por ejemplo en el área financiera
-Clasificación de reclamos de usuarios en empresas de servicios públicos.
-Monitoreo de redes sociales para campañas políticas.
-Automatización de análisis de encuestas de satisfacción.

---

---

## Reflexiones Finales del Curso

Completaste los cuatro ejercicios de NLP con Transformers, incluyendo un desafío autónomo. Vamos a reflexionar sobre lo aprendido:

### Ejercicio 1 - Moderación de Comentarios
- **Aprendiste:** A clasificar texto usando modelos de análisis de sentimientos
- **Aplicación:** Moderación automática de redes sociales, priorización de tickets de soporte
- **Limitaciones:** Los modelos pueden tener problemas con ironía, sarcasmo o lenguaje muy coloquial

### Ejercicio 2 - Extracción de CVs
- **Aprendiste:** A extraer entidades nombradas (personas, organizaciones, lugares) de texto
- **Aplicación:** Automatización de RRHH, análisis de documentos, extracción de información estructurada
- **Limitaciones:** Algunos modelos pueden confundir entidades o no detectar nombres poco comunes

### Ejercicio 3 - Chatbot de Soporte
- **Aprendiste:** A usar Question Answering para responder preguntas basadas en un contexto
- **Aplicación:** Chatbots, asistentes virtuales, sistemas de FAQ automáticas
- **Limitaciones:** El modelo solo puede responder sobre información presente en el contexto

### Ejercicio 4 - Desafío Autónomo
- **Aprendiste:** A combinar múltiples técnicas de NLP para resolver un problema real completo
- **Aplicación:** Sistemas integrados de análisis de feedback, gestión de reputación online
- **Habilidad clave:** Autonomía para investigar, implementar y evaluar soluciones de NLP

### Próximos pasos sugeridos

1. **Explorá más modelos** en [Hugging Face Hub](https://huggingface.co/models)
2. **Combiná técnicas:** Por ejemplo, usá clasificación de sentimientos + QA para un chatbot más inteligente
3. **Experimentá con otros idiomas** o dialectos regionales
4. **Investigá fine-tuning:** Aprendé a ajustar modelos con tus propios datos
5. **Desarrollá un proyecto propio:** Elegí un problema real que te interese y aplicá estas técnicas

---

## Glosario Técnico

### Conceptos fundamentales

**Transformer**  
Arquitectura de red neuronal basada en mecanismos de atención que revolucionó el NLP en 2017. Permite procesar secuencias de texto completas simultáneamente en lugar de palabra por palabra.

**Pipeline**  
Interfaz de alto nivel en Hugging Face que encapsula todo el proceso de preprocesamiento, inferencia y postprocesamiento de un modelo. Facilita el uso de modelos preentrenados con pocas líneas de código.

**Modelo preentrenado**  
Modelo de machine learning que fue entrenado previamente con grandes cantidades de datos. Puede usarse directamente o ajustarse (fine-tuning) para tareas específicas.

**Tokenización**  
Proceso de dividir texto en unidades más pequeñas (tokens) que el modelo puede procesar. Puede ser a nivel de palabras, subpalabras o caracteres.

### Tareas de NLP

**Text Classification (Clasificación de texto)**  
Tarea de asignar una o más etiquetas a un texto. Incluye análisis de sentimientos, detección de spam, clasificación de temas, etc.

**Sentiment Analysis (Análisis de sentimientos)**  
Subtipo de clasificación que identifica la polaridad emocional de un texto (positivo, negativo, neutral). Se usa en redes sociales, reseñas de productos, atención al cliente.

**Named Entity Recognition - NER (Reconocimiento de entidades nombradas)**  
Tarea de identificar y clasificar nombres propios en texto: personas (PER), organizaciones (ORG), ubicaciones (LOC), fechas, cantidades monetarias, etc.

**Question Answering - QA (Respuesta a preguntas)**  
Tarea de responder preguntas en lenguaje natural basándose en un contexto dado. El modelo extrae la respuesta directamente del texto proporcionado.

**Text Generation (Generación de texto)**  
Tarea de crear texto nuevo de manera coherente a partir de un prompt inicial. Incluye completado de texto, escritura creativa, chatbots conversacionales.

**Summarization (Resumen automático)**  
Tarea de condensar un texto largo en una versión más corta manteniendo la información más importante.

**Translation (Traducción automática)**  
Tarea de traducir texto de un idioma a otro usando modelos de secuencia a secuencia.

### Componentes técnicos

**Embedding (Representación vectorial)**  
Representación numérica de palabras o tokens como vectores en un espacio multidimensional. Palabras con significados similares tienen embeddings cercanos.

**Attention (Atención)**  
Mecanismo que permite al modelo enfocarse en diferentes partes del input al procesar cada elemento. Es el componente clave de la arquitectura Transformer.

**Fine-tuning (Ajuste fino)**  
Proceso de tomar un modelo preentrenado y entrenarlo adicionalmente con datos específicos de tu dominio para mejorar su desempeño en tu tarea particular.

**Inference (Inferencia)**  
Proceso de usar un modelo ya entrenado para hacer predicciones sobre datos nuevos. No implica entrenamiento, solo aplicación del modelo.

**Score / Confidence (Puntuación / Confianza)**  
Valor numérico (generalmente entre 0 y 1) que indica qué tan seguro está el modelo de su predicción. Valores más altos indican mayor confianza.

### Modelos mencionados

**BETO**  
Versión de BERT (Bidirectional Encoder Representations from Transformers) entrenada específicamente con texto en español. Usado en análisis de sentimientos y otras tareas de clasificación.

**RoBERTa**  
Variante optimizada de BERT con mejoras en el proceso de preentrenamiento. Usado para múltiples tareas de NLP en español.

**BERT (Bidirectional Encoder Representations from Transformers)**  
Modelo Transformer que lee texto bidireccionalmente (izquierda a derecha y derecha a izquierda simultáneamente) para comprender mejor el contexto.

**GPT (Generative Pre-trained Transformer)**  
Familia de modelos diseñados específicamente para generación de texto. Leen texto de izquierda a derecha y predicen la siguiente palabra.

### Plataformas y librerías

**Hugging Face**  
Plataforma y empresa que desarrolla herramientas de NLP de código abierto. Su librería Transformers es el estándar de facto para trabajar con modelos de lenguaje.

**Hugging Face Hub**  
Repositorio online con miles de modelos preentrenados, datasets y demos interactivas. Permite compartir y descargar modelos fácilmente.

**PyTorch / TensorFlow**  
Frameworks de deep learning usados como backend por la librería Transformers. PyTorch es más común en investigación, TensorFlow en producción.

### Métricas y evaluación

**Label (Etiqueta)**  
Categoría asignada por el modelo a un texto. En análisis de sentimientos: POS (positivo), NEG (negativo), NEU (neutral).

**Aggregation strategy (Estrategia de agregación)**  
En NER, método para combinar tokens que pertenecen a la misma entidad. Por ejemplo, "Buenos" y "Aires" se agrupan en una sola entidad "Buenos Aires".

**Context (Contexto)**  
En QA, el documento o párrafo que contiene la información necesaria para responder la pregunta. El modelo busca la respuesta dentro de este contexto.

---

## Recursos adicionales

- **Documentación oficial de Transformers:** https://huggingface.co/docs/transformers
- **Modelos en español:** https://huggingface.co/models?language=es
- **Curso gratuito de Hugging Face:** https://huggingface.co/course
- **Comunidad en español:** https://huggingface.co/spaces

---

*Este cuaderno fue diseñado con fines educativos para estudiantes de NLP en Argentina.*